# Setup

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

device = 'cuda'

# Load data

In [ ]:
orig_train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
orig_train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
# train_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
orig_test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
orig_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
train_features = orig_train_features.copy()
train_targets_scored = orig_train_targets_scored.copy()
test_features = orig_test_features.copy()
submission = orig_submission.copy()

# Gene and Cells

In [ ]:
genes = [feature for feature in train_features.columns if feature.startswith('g-')]
cells = [feature for feature in train_features.columns if feature.startswith('c-')]

print(f'Number of gene features: {len(genes)}')
print(f'Number of cell features: {len(cells)}')

In [ ]:
def correlationPlot(df, title):
    f = plt.figure(figsize=(25, 25))
    plt.matshow(df.corr(), fignum=f.number)
    plt.xticks(range(df.shape[1]), df.columns, fontsize=8, rotation=45)
    plt.yticks(range(df.shape[1]), df.columns, fontsize=8)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=12)
    plt.title(title, fontsize=16)
    plt.show()

In [ ]:
correlationPlot(train_features[genes], 'Genes Correlation Matrix')

In [ ]:
correlationPlot(train_features[cells], 'Cells Correlation Matrix')

# Data preparation

In [ ]:
# remove ctl_vehicles
ctl_vehicle_ids_index = train_features[train_features['cp_type'] == 'ctl_vehicle'].index
train_features.drop(ctl_vehicle_ids_index, inplace=True)
train_targets_scored.drop(ctl_vehicle_ids_index, inplace=True)

# drop sig_id
train_features.drop('sig_id', axis=1, inplace=True)
train_targets_scored.drop('sig_id', axis=1, inplace=True)
test_features.drop('sig_id', axis=1, inplace=True)

# cp_dose
train_features['cp_dose'] = train_features['cp_dose'].map({'D1':0, 'D2':1})
test_features['cp_dose'] = test_features['cp_dose'].map({'D1':0, 'D2':1})

# cp_time
train_features['cp_time'] = train_features['cp_time']//24-1
test_features['cp_time'] = test_features['cp_time']//24-1

# drop cp_type
train_features.drop('cp_type', axis=1, inplace=True)
test_features.drop('cp_type', axis=1, inplace=True)

In [ ]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scld_train_features = scaler.fit_transform(train_features)
scld_test_features = scaler.transform(test_features)

In [ ]:
# Create validation set
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(scld_train_features, train_targets_scored,
                                                      train_size=.8, random_state=42)

y_train = y_train.values
y_valid = y_valid.values

# PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(scld_train_features)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.94) + 1

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(cumsum, linewidth=3)
plt.axis([0, 800, 0, 1])
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")
plt.plot([d, d], [0, 0.95], "k:")
plt.plot([0, d], [0.95, 0.95], "k:")
plt.plot(d, 0.95, "ko")
plt.annotate("~500 Dimensions", xy=(500, 0.94), xytext=(420, 0.73),
             arrowprops=dict(arrowstyle="->"), fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
pca = PCA(n_components=0.94)
X_reduced_pca = pca.fit_transform(scld_train_features)

X_pca_train, X_pca_valid = train_test_split(X_reduced_pca, train_size=.8, random_state=42)

# Loss Function

In [ ]:
def logloss(y_pred, Y_label):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    cross_entropy = -(Y_label * np.log(y_pred) - (1-Y_label) * np.log(1-y_pred)).mean()
    return cross_entropy

# History

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)

def runNN(model, X_train=X_train, X_valid=X_valid):
    return model.fit(X_train, y_train, epochs=1000,
                  validation_data=(X_valid, y_valid),
                  callbacks=[early_stopping_cb])

# Graph Accuracy

In [ ]:
def graphAcc(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
    plt.show()

# Submission File

In [ ]:
sig_ids = submission.sig_id

def submit(submit):
    submission.drop(columns = ['sig_id'], inplace = True)
    submission[:] = test_predict
    submission.insert(0, "sig_id", sig_ids, True)
    submission.to_csv('submission.csv', index = False)
    print('File created')

# Base Model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

valid_pred = lin_reg.predict(X_valid)

logloss(valid_pred, y_valid)

# Basic NN

In [ ]:
inputUnits = X_train.shape[1:]
outputUnits = y_train.shape[1:][0]

In [ ]:
def get_compiled_model():
    model = keras.Sequential([
        keras.layers.Dense(10000, activation='relu', input_shape=inputUnits),
        keras.layers.Dense(5000, activation='relu'),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dense(outputUnits, activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()

In [ ]:
history = runNN(nn)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_valid, y_valid)

# Base NN With Dropout

In [ ]:
def get_compiled_model():
    model = keras.Sequential([
        keras.layers.Dense(10000, activation='relu', input_shape=inputUnits),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(5000, activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(outputUnits, activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()

history = runNN(nn)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_valid, y_valid)

# Base NN Adjusted Dropout

In [ ]:
def get_compiled_model():
    model = keras.Sequential([
        keras.layers.Dense(10000, activation='relu', input_shape=inputUnits),
        keras.layers.Dropout(0.05),
        keras.layers.Dense(5000, activation='relu'),
        keras.layers.Dropout(0.05),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dropout(0.05),
        keras.layers.Dense(outputUnits, activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()
    
history = runNN(nn)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_valid, y_valid)

# Smaller NN

In [ ]:
def get_compiled_model():
    model = keras.Sequential([
        keras.layers.Dense(1000, activation='relu', input_shape=inputUnits),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(outputUnits, activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()
    
history = runNN(nn)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_valid, y_valid)

# Smaller NN and Learning Rate

In [ ]:
def get_compiled_model():
    model = keras.Sequential([
        keras.layers.Dense(1000, activation='relu', input_shape=inputUnits),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(outputUnits, activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.00005)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()
    
history = runNN(nn)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_valid, y_valid)

# Smaller NN With Dropout

In [ ]:
def get_compiled_model():
    model = keras.Sequential([
        keras.layers.Dense(1000, activation='relu', input_shape=inputUnits),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(outputUnits, activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()

history = runNN(nn)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_valid, y_valid)

# Wide and Deep

In [ ]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=1000, activation='relu'):
        super().__init__()
        self.hidden1 = keras.layers.Dense(units, activation=activation, input_shape=inputUnits)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.hidden3 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(outputUnits, activation='softmax')
        self.aux_output = keras.layers.Dense(outputUnits, activation='softmax')
        
    def call(self, X):
        hidden1 = self.hidden1(X)
        hidden2 = self.hidden2(hidden1)
        hidden3 = self.hidden3(hidden2)
        concat = keras.layers.concatenate([X, hidden3])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden3)
        return main_output, aux_output

def get_compiled_model():
    model = WideAndDeepModel(500, activation='relu')
    
    optimizer = keras.optimizers.Adam(lr=1e-3)
    
    model.compile(loss='binary_crossentropy',
                  loss_weights=[0.9, 0.1],
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()
    
history = runNN(nn)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_valid, y_valid)

# NN With PCA

In [ ]:
inputUnitsPCA = X_pca_train.shape[1:]

def get_compiled_model():
    model = keras.Sequential([
        keras.layers.Dense(10000, activation='relu', input_shape=inputUnitsPCA),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(5000, activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(outputUnits, activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()

history = runNN(nn, X_pca_train, X_pca_valid)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_pca_valid, y_valid)

# Large NN and Dropout to Small NN and Dropout

In [ ]:
def get_compiled_model():
    model = keras.Sequential([
        keras.layers.Dense(10000, activation='relu', input_shape=inputUnits),
        keras.layers.Dropout(0.9),
        keras.layers.Dense(10000, activation='relu'),
        keras.layers.Dropout(0.05),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dropout(0.001),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(outputUnits, activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.0001)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

with tf.device("/GPU:0"):
    nn = get_compiled_model()

history = runNN(nn)

In [ ]:
graphAcc(history)

In [ ]:
nn.evaluate(X_valid, y_valid)

In [ ]:
test_predict = nn.predict(scld_test_features)
submit(test_predict)